In [12]:
!pip install pinecone-client cohere

In [13]:
# %% Load PDF
from PyPDF2 import PdfFileReader
from PyPDF2 import PdfReader

from io import BytesIO
# import fitz  # PyMuPDF

def load_pdf(pdf_path):
    # with open(pdf_path, "rb") as file:
    pdf = PdfReader(pdf_path)
    text = ""
    for page in pdf.pages:
        text += page.extract_text() or ""  # Extract text from each page
        # print("extracted text ",text)
    return text

In [14]:
# %% Get Embeddings
import cohere

co = cohere.Client('brV6x6dombNlECLz3031MhwM2UxHom1Gx2mp0BX8')

def get_embeddings(text):
    response = co.embed(texts=[text])
    embeddings = response.embeddings[0]
    print("Dimensions of the embeddings:", len(embeddings))
    return embeddings

In [15]:
# Initialize Pinecone client
# pinecone_client = pinecone.Client(api_key="564a623c-4d0a-4247-8f53-1e171c26b4e2")
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key='564a623c-4d0a-4247-8f53-1e171c26b4e2')
# index_name = "quickstart"

# from pinecone.grpc import PineconeGRPC as Pinecone

# pc = Pinecone(api_key="564a623c-4d0a-4247-8f53-1e171c26b4e2")
# index = pc.Index("example-index")
# index = pinecone_client.Index(index_name)

In [17]:
import pinecone
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="564a623c-4d0a-4247-8f53-1e171c26b4e2")
index_name = "quickstart2"

# # if not pc.has_index(index_name):
# pc.create_index(
#     name=index_name,
#     dimension=4096,
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud='aws', 
#         region='us-east-1'
#     ) 
#     ) 




In [16]:
# pc.delete_index(index_name)

In [67]:
# pc.delete_index('quickstart')

In [18]:
index = pc.Index(index_name)

In [19]:
def store_embeddings(id, embedding, text):
    # Add the text to metadata while storing the embeddings
    index.upsert([(id, embedding, {"text": text})])


In [20]:
def retrieve_documents(query_embedding):
    results = index.query(vector=query_embedding, top_k=5,
                          include_values=True,      # Include vector values in the response
                          include_metadata=True )   # Include metadata in the response
    return results['matches']


In [21]:
def generate_answer(question, context):
    response = co.generate(prompt=f"Question: {question}\nContext: {context}\nAnswer:", max_tokens=100)
    return response.generations[0].text


In [22]:
# Function to split the text into sections of 150 words
def split_into_sections(text, section_size=150):
    words = text.split()
    sections = [' '.join(words[i:i+section_size]) for i in range(0, len(words), section_size)]
    return sections

# Modify the store_embeddings function to store multiple sections
def store_embeddings_in_sections(document_id, document_text, section_size=150):
    sections = split_into_sections(document_text, section_size)
    
    for idx, section in enumerate(sections):
        section_id = f"{document_id}_section_{idx+1}"  # Create a unique ID for each section
        section_embedding = get_embeddings(section)  # Get the embedding for the section
        index.upsert([(section_id, section_embedding, {'text': section})])  # Store with metadata



In [73]:
# Example workflow
query = "What is the main topic of the document?"
document_text = load_pdf('my_pdf.pdf')

# Store the document's sections in Pinecone
store_embeddings_in_sections("document_id", document_text, section_size=150)

Dimensions of the embeddings: 4096
Dimensions of the embeddings: 4096
Dimensions of the embeddings: 4096


In [78]:
# print(document_text)

In [79]:
# document_embedding

In [11]:
query = "how to use docker in this required context ? keep it brisk"
query_embedding = get_embeddings(query)
retrieved_segments = retrieve_documents(query_embedding)
# print(retrieved_segments)
# print(document_text)
# Assuming retrieved_segments is a list of segments
context = " ".join([segment['metadata'].get('text', '') for segment in retrieved_segments])

# Print the context to verify
print("Section of text :", context)


# Generate an answer using the query and context
answer = generate_answer(query, context)
print("Answer:", answer)
# print(context)
# answer = generate_answer(query, context)
# print("Answer:", answer)
# for segment in retrieved_segments:
#     print(segment)

Dimensions of the embeddings: 4096
Section of text : interact with the bot. ● Documentation on how the user can upload files, ask questions, and view the bot's responses. ● Example interactions demonstrating the bot's capabilities. Guidelines: ● Use Docker to containerize the application for easy deployment. ● Ensure the system can handle large documents and multiple queries without significant performance drops. ● Share the code, deployment instructions, and the final working model through GitHub. General Guidelines: 1. Ensure modular and scalable code following best practices for both frontend and backend development. 2. Document your approach thoroughly, explaining your decisions, challenges faced, and solutions. 3. Provide a detailed ReadMe file in your GitHub repository, including setup and usage instructions. 4. Submissions should include: ○ Source code for both the notebook and the interface. ○ A fully functional Colab notebook. ○ Documentation on the pipeline and deployment ins